<a href="https://colab.research.google.com/github/Rmm6699/HW/blob/master/W5_SOTA_Tabular_Data_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import argparse
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

from utils import preprocess_data

In [ ]:
#set random seed
seed = 1

# Load Data
X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)
print(f'Training Data shape: {X_train.shape},\
        Testing Data shape: {X_test.shape},\
        num of class: {max(y_train.max(), y_test.max())+1}')

Training Data shape: (1347, 64),        Testing Data shape: (450, 64),        num of class: 10


# Deep Forest

Zhi-Hua Zhou, Ji Feng, Deep Forest, 2020.

[Github](https://github.com/LAMDA-NJU/Deep-Forest)

In [ ]:
!pip3 install deep-forest

In [ ]:
from deepforest import CascadeForestClassifier

model = CascadeForestClassifier(random_state=seed)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred) * 100
print("\nTesting Accuracy: {:.3f} %".format(acc))

[2022-05-20 04:34:54.589] Start to fit the model:
[2022-05-20 04:34:54.589] Fitting cascade layer = 0 
[2022-05-20 04:34:56.760] layer = 0  | Val Acc = 92.450 % | Elapsed = 2.171 s
[2022-05-20 04:34:56.778] Fitting cascade layer = 1 
[2022-05-20 04:34:58.683] layer = 1  | Val Acc = 92.850 % | Elapsed = 1.905 s
[2022-05-20 04:34:58.698] Fitting cascade layer = 2 
[2022-05-20 04:35:00.557] layer = 2  | Val Acc = 92.950 % | Elapsed = 1.859 s
[2022-05-20 04:35:00.569] Fitting cascade layer = 3 
[2022-05-20 04:35:02.484] layer = 3  | Val Acc = 92.850 % | Elapsed = 1.915 s
[2022-05-20 04:35:02.485] Early stopping counter: 1 out of 2
[2022-05-20 04:35:02.498] Fitting cascade layer = 4 
[2022-05-20 04:35:04.367] layer = 4  | Val Acc = 92.600 % | Elapsed = 1.869 s
[2022-05-20 04:35:04.367] Early stopping counter: 2 out of 2
[2022-05-20 04:35:04.367] Handling early stopping
[2022-05-20 04:35:04.368] The optimal number of layers: 3
[2022-05-20 04:35:04.371] Start to evalute the model:
[2022-05-20

# VIME
Jinsung Yoon, Yao Zhang, James Jordon, Mihaela van der Schaar, VIME: Extending the Success of Self- and Semi-supervised Learning to Tabular Domain, Neural Information Processing Systems (NeurIPS), 2020.

[Github](https://github.com/jsyoon0823/VIME)

In [ ]:
from vime.vime_utils import perf_metric
from vime.model import VIME

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
# Set arguments
parser = argparse.ArgumentParser()
parser.add_argument('--p_m', type=float, default=0.3)
parser.add_argument('--alpha', type=float, default=2.0)
parser.add_argument('--K', type=int, default=3)
parser.add_argument('--beta', type=float, default=2.0)
parser.add_argument('--hidden_dim', type=int, default=128)
parser.add_argument('--label_data_rate', type=float, default=0.9)
parser.add_argument('--file_path', type=str, default='./vime/save_model')
parser.add_argument('--file_name', type=str, default='./vime/save_model/encoder_model.h5')
args = parser.parse_args([])

# Initial VIME Model
vime = VIME(args)

# Preprocess Data
X_train_vime, y_train_vime, X_unlab_vime, X_test_vime = preprocess_data(X_train, y_train, X_test, y_test, 
                                                                        Normalize=True, label_data_rate=args.label_data_rate, one_hot=True)

# Start train the vime-self model only using X_unlab_vime (unsupervised)
vime.train_self_model_and_save(X_unlab_vime, batch_size=128, epochs=10)

# Train the vime-semi model using X_train_vime, y_train_vime and X_unlab_vime (semi-supervised)
y_test_pred = vime.train_semi_model_and_predict(X_train_vime, y_train_vime, 
                                                X_unlab_vime, X_test_vime, batch_size=128, epochs=1000)

Train on 135 samples
Epoch 1/10
135/135 [==============================] - 0s 628us/sample - loss: 1.0700 - mask_loss: 0.6970 - feature_loss: 0.1823
Epoch 2/10
135/135 [==============================] - 0s 22us/sample - loss: 1.0293 - mask_loss: 0.6753 - feature_loss: 0.1710
Epoch 3/10
135/135 [==============================] - 0s 15us/sample - loss: 1.0006 - mask_loss: 0.6617 - feature_loss: 0.1683
Epoch 4/10
135/135 [==============================] - 0s 15us/sample - loss: 0.9724 - mask_loss: 0.6494 - feature_loss: 0.1604
Epoch 5/10
135/135 [==============================] - 0s 22us/sample - loss: 0.9418 - mask_loss: 0.6298 - feature_loss: 0.1540
Epoch 6/10
135/135 [==============================] - 0s 15us/sample - loss: 0.9100 - mask_loss: 0.6116 - feature_loss: 0.1455
Epoch 7/10
135/135 [==============================] - 0s 15us/sample - loss: 0.8758 - mask_loss: 0.5957 - feature_loss: 0.1395
Epoch 8/10
135/135 [==============================] - 0s 22us/sample - loss: 0.8430 - mas

In [ ]:
# Test VIME
acc = accuracy_score(y_test, np.argmax(y_test_pred, -1))

print('VIME accuracy: '+ str(acc))

VIME accuracy: 0.9711111111111111


# SubTab
Talip Ucar, Ehsan Hajiramezanali, Lindsay Edwards, SubTab: Subsetting Features of Tabular Data for Self-Supervised Representation Learning, Neural Information Processing Systems (NeurIPS), 2021.

[Github](https://github.com/AstraZeneca/SubTab)

In [ ]:
import torch
from subtab.model import SubTab
from subtab.utils.arguments import get_arguments

In [ ]:
# Set arguments
args = get_arguments(X_train)

# Preprocess Data
X_train_sub, y_train_sub, X_test_sub, y_test_sub = preprocess_data(X_train, y_train, X_test, y_test,
                                                                   Normalize=True, one_hot=False)

Directories are set.


In [ ]:
model = SubTab(args)
model.fit(X_train_sub, y_train_sub, epochs=30)

Directories are set.
Building the models for training and evaluation in SubTab framework...
----------------------------------------Summary of the models (an Autoencoder and Projection network):----------------------------------------
==================================AE Model==================================
AEWrapper(
  (encoder): ShallowEncoder(
    (hidden_layers): HiddenLayers(
      (layers): ModuleList(
        (0): Linear(in_features=28, out_features=784, bias=True)
        (1): LeakyReLU(negative_slope=0.01)
      )
    )
  )
  (decoder): ShallowDecoder(
    (first_layer): Linear(in_features=784, out_features=64, bias=True)
  )
  (linear_layer1): Linear(in_features=784, out_features=784, bias=True)
  (linear_layer2): Linear(in_features=784, out_features=784, bias=True)
)



Losses per batch - Total:8.3149, X recon:4.1751, contrastive:4.1380, z distance:0.001861, Progress: 100%|█| 33/33 [00:1
Epoch-1 Total training loss:9.7469 | Losses per batch - X recon:3.7700, contrastive:4.0597, z distance:0.033281, Progre
Epoch-2 Total training loss:8.0440 | Losses per batch - X recon:3.8267, contrastive:4.0815, z distance:0.061836, Progre
Epoch-3 Total training loss:7.7445 | Losses per batch - X recon:4.0535, contrastive:3.9625, z distance:0.064687, Progre
Epoch-4 Total training loss:7.5664 | Losses per batch - X recon:3.4365, contrastive:3.9848, z distance:0.077123, Progre
Epoch-5 Total training loss:7.4884 | Losses per batch - X recon:3.5386, contrastive:3.9495, z distance:0.088595, Progre
Epoch-6 Total training loss:7.4442 | Losses per batch - X recon:3.3210, contrastive:3.9901, z distance:0.046854, Progre
Epoch-7 Total training loss:7.3628 | Losses per batch - X recon:3.4508, contrastive:3.9924, z distance:0.046222, Progre
Epoch-8 Total training loss:7.3400 | Los

Done with saving models.


In [ ]:
# Get embedding
z_train_sub, y_train_embed = model.get_embedding()

--encoder is loaded
Done with loading models.


100%|█████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 135.05it/s]


In [ ]:
model.evaluate(X_test_sub, y_test_sub, z_train_sub, y_train_embed)

100%|█████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 134.87it/s]


******************** Running evaluation using Logistic Regression trained on the joint embeddings of training set and tested on that of test set********************
>>>>>>>>>>Sweeping C parameter. Smaller C values specify stronger regularization:
**********C=0.01**********
Training score: 0.6638257575757576
Test score: 0.6644444444444444
**********C=0.1**********
Training score: 0.8039772727272727
Test score: 0.7777777777777778
**********C=1**********
Training score: 0.8920454545454546
Test score: 0.8288888888888889
**********C=10**********
Training score: 0.9810606060606061
Test score: 0.8555555555555555
**********C=100.0**********
Training score: 1.0
Test score: 0.8555555555555555
**********C=1000.0**********
Training score: 1.0
Test score: 0.8511111111111112
**********C=10000.0**********
Training score: 1.0
Test score: 0.8466666666666667
**********C=100000.0**********
Training score: 1.0
Test score: 0.8444444444444444
**********C=1000000.0**********
Training score: 1.0
Test score: 0